In [1]:
!pip install transformers==4.44.2

In [1]:
import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import pandas as pd

In [2]:
datos=pd.read_csv("dataset_dialogos/dataset_dialogos.csv")

In [4]:
datos

,source_file,dialogue
0,130000.jsonl,Está saliendo la luna.\nNo podemos perder tiempo.
1,130000.jsonl,Está descansando.\nEsperando una nueva vida.
2,130000.jsonl,No.\nVamos. No puede hacerte nada.
3,130000.jsonl,Tiene el cuello roto.\nEl cerebro no sirve.
4,130000.jsonl,"Gracias, caballeros.\nLa clase ha terminado."
...,...,...
19983467,89999.jsonl,¿De veras?\nSí.
19983468,89999.jsonl,"Ah, sí.\nSí."
19983469,89999.jsonl,¿Entonces me amas un poco?\nNo.
19983470,89999.jsonl,"Si te beso, ¿me dispararân?\nNo."


In [3]:
datos_preprocesado=datos[50:100]["dialogue"]
datos_preprocesado

50    Enciérralo en el sótano.\n¡Mátelo! Es un monst...
51    ¡Déjalo ya!\nNos echarás encima a toda la coma...
52       Sal de aquí.\nTiene la fuerza de diez hombres.
53             Odiaba a Fritz.\nSiempre lo martirizaba.
54    Vamos, recupere la compostura.\n¿Qué podemos h...
55    ¿La tiene?\nSí, aquí está.\nEs muy fuerte. Med...
56                                         ¿Listo?\nSí.
57      ¿Le ha hecho daño?\nNo, estoy bien. No es nada.
58                                No.\nDemasiado tarde.
59    No quiero que lo vean.\nRápido, échenme una ma...
60           ¿Es ésta la puerta principal?\nSí, así es.
61              Está bien.\nNo me gusta, pero allá voy.
62    ¿lntenta provocar un incendio?\n¿Para qué es eso?
63    ¿Qué le pasa? Parece como si le hubiera cocead...
64    ¿Dónde estás, querida?\nVaya, conque estás ahí...
65    Déjelos solos.\nNo hay barandilla ni nada.\n¿C...
66                                   Henry.\nElizabeth.
67    ¡Henry!\n¡Victor! ¡Dr. Waldman!\nVengan en

In [6]:
#Prueba del tokenizer y embeddings model

In [9]:
from transformers import BertTokenizer,BertModel
model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
texto = """ "a ¿qué hora "pasa el, ciento siete.   """
tokens = tokenizer.tokenize(texto)
print(tokens)
# ['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
tokens_id=tokenizer.convert_tokens_to_ids(tokens)
#[169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
embedding_vector = model.embeddings.word_embeddings.weight
embedding_vector 
#embedding_vector.shape

['"', 'a', '¿', 'qué', 'hora', '"', 'pasa', 'el', ',', 'cien', '##to', 'siete', '.']


Parameter containing:
tensor([[ 0.0260, -0.0062, -0.0041,  ...,  0.0297,  0.0242,  0.0197],
        [ 0.0104, -0.0136,  0.0067,  ...,  0.0124,  0.0267,  0.0337],
        [ 0.0221, -0.0036,  0.0193,  ...,  0.0069,  0.0268,  0.0050],
        ...,
        [ 0.0068,  0.0189,  0.0267,  ...,  0.0229,  0.0647,  0.0437],
        [ 0.0184,  0.0148,  0.0243,  ...,  0.0321,  0.0071,  0.0204],
        [ 0.0214,  0.0188, -0.0134,  ..., -0.0060,  0.0058, -0.0059]],
       requires_grad=True)

In [14]:
#Funciones de etiquetado
from typing import List, Union
from transformers import BertTokenizer,BertModel

model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def separar_palabras(texto:str)->List[str]:
    primera_letra=0
    palabras=[]
    
    #palabras que terminan por espacio
    for i in range(len(texto)):
        if texto[i] == " ":
            palabra=texto[primera_letra:i]
            primera_letra=i+1
            palabras.append(palabra)
        #palabras que terminan por cambio de parrafo
        if texto[i]=="\n":
            palabra=texto[primera_letra:i]
            primera_letra=i+1  
            palabras.append(palabra)

        if texto[i]=="¡" or texto[i]=='"':
            primera_letra+=1
    
    #agrego ultima palabra
    palabras.append(texto[primera_letra:])
    return palabras

def clasificar_mayusculas(palabra:str)->int:
    """Toma una palabra y cuenta sus mayusculas"""
    cantidad=0
    for i in range(len(palabra)):
        if palabra[i].lower()!=palabra[i]:
            cantidad+=1
    res=0
    if cantidad==len(palabra): #todas son mayuscula
        res=3
    elif cantidad==1 and palabra[0].lower()!=palabra[0]: #solo la primera es mayuscula
        res=1
    elif cantidad>0: #alguna es mayuscula (iPhone tiene que estar aca)
        res=2
    else: #Ninguna de las anteriores
        res=0
    return res
    
def asignar_labels(datos: pd.Series, path=None)->pd.DataFrame:
    punt_inicial=[]
    puntos=[]
    cierre_pregunta=[]
    comas=[]
    instancias=[]
    token_ids=[]
    tokens=[]
    capitalizacion=[]
    instancia=0
    print("Comenzando procesamiento")
    for dialogo in datos:
        if instancia % (len(datos)//10)==0:
            print(f"procesando un {instancia/len(datos)*100}%")
            
        palabras_originales= separar_palabras(dialogo)
        palabras_lower= [palabra.lower() for palabra in palabras_originales]
        instancia+=1
        #print(palabras_lower)

        punt_inicial_seguridad=[]
        puntos_seguridad=[]
        cierre_pregunta_seguridad=[]
        comas_seguridad=[]
        instancias_seguridad=[]
        capitalizacion_seguridad=[]
        palabra_tokenizada_ids_seguridad=[]
        tokens_seguridad=[]
        es_seguro=True
        # hola, ¿Como estas. ? ### hola, ; ¿Como ; estas?
        #token  hola ,["¿", como] estas ?
        for idx, palabra in enumerate(palabras_lower):
            
            hay_signo_apertura=False 
            palabra_tokenizada= tokenizer.tokenize(palabra)

            #Recordartorio SON PALABRAS. ejemplos: '¿Que' o "escuela?" o "Franquito," o "Emi."  "¿ que ?"
            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[0]=="¿":
                    palabra_tokenizada=palabra_tokenizada[1:]
                    hay_signo_apertura=True
                    punt_inicial_seguridad.append(1) # ¿
                
             
            if len(palabra_tokenizada)>0:
                #estas ?
                if palabra_tokenizada[-1]=="?":        
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    
                    #relleno las demas listas
                    puntos_seguridad+=[0]*(len(palabra_tokenizada))
                    comas_seguridad+=[0]*(len(palabra_tokenizada))

                    
                    cierre_pregunta_seguridad+=[0]*(len(palabra_tokenizada)-1) 
                    cierre_pregunta_seguridad.append(1) #?
                    
                elif palabra_tokenizada[-1]==".":
                    
                    palabra_tokenizada=palabra_tokenizada[:-1]

                    comas_seguridad+=[0]*(len(palabra_tokenizada))
                    cierre_pregunta_seguridad+=[0]*(len(palabra_tokenizada))
                    
                    puntos_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    puntos_seguridad.append(1) #.
        
                elif palabra_tokenizada[-1]==",":
                    palabra_tokenizada=palabra_tokenizada[:-1]

                    cierre_pregunta_seguridad+=[0]*(len(palabra_tokenizada))
                    puntos_seguridad+=[0]*(len(palabra_tokenizada))

                    comas_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    comas_seguridad.append(1) #,
        
                else:
                    puntos_seguridad+=[0]*(len(palabra_tokenizada))
                    cierre_pregunta_seguridad+=[0]*(len(palabra_tokenizada))
                    comas_seguridad+=[0]*(len(palabra_tokenizada))
                    

            
            palabra_tokenizada_ids= tokenizer.convert_tokens_to_ids(palabra_tokenizada) 
            
            if hay_signo_apertura:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada)-1)
                
            else:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada))
            
            #capitalizacion
            clasificacion=None
            palabra_orginal= palabras_originales[idx]
            if len(palabra_tokenizada)>0:
                if palabra_orginal[0]=="¿" or palabra_orginal[0]=='"' or palabra_orginal[0]=='¡':
                    if palabra_orginal[-1]=='"' or palabra_orginal[-1]=='!' or palabra_orginal[-1]=='?':
                        clasificacion= clasificar_mayusculas(palabra_orginal[1:-1])
                    else: 
                         clasificacion= clasificar_mayusculas(palabra_orginal[1:])
                else:   
                    clasificacion= clasificar_mayusculas(palabra_orginal)

            if len(palabra_tokenizada)==0:
                es_seguro=False 
                break
            
            capitalizacion_seguridad+= [clasificacion]*len(palabra_tokenizada)
            palabra_tokenizada_ids_seguridad+=palabra_tokenizada_ids
            tokens_seguridad+=palabra_tokenizada
            instancias_seguridad+=[instancia]*len(palabra_tokenizada)
        
        if es_seguro:
            capitalizacion+=capitalizacion_seguridad
            token_ids+= palabra_tokenizada_ids_seguridad
            tokens+= tokens_seguridad
            instancias+=instancias_seguridad
            punt_inicial+=punt_inicial_seguridad
            puntos+=puntos_seguridad
            comas+=comas_seguridad
            cierre_pregunta+=cierre_pregunta_seguridad
            
            if len(capitalizacion)!=len(punt_inicial): #tamaño cantidad de token procesados
                print("dif inicial",instancia,dialogo)
                return
            if len(capitalizacion)!=len(puntos):
                print("dif final",instancia,dialogo)
                print(len(puntos))
                print(len(comas))
                print(len(cierre_pregunta))
                return
                
        else:
            instancia-=1
    
    
        if len(capitalizacion)!=len(punt_inicial):
            print("dif inicial",instancia)
            print(len(capitalizacion))
            print(len(punt_inicial))
            
        if len(capitalizacion)!=len(comas):
            print("dif final",instancia)
            print(len(puntos))
            print(len(comas))
            print(len(cierre_pregunta))
    
    datos_listos= pd.DataFrame({"instancia_id":instancias,"token":tokens,"token_id":token_ids,"punt_inicial":punt_inicial,"capitalizacion":capitalizacion,"puntos":puntos,"comas":comas,"cierre_pregunta":cierre_pregunta})    

    if path!=None:
        datos_listos.to_csv(path,index=False)
    return datos_listos    
            

datos_listos=asignar_labels(datos_preprocesado,"datos_modelo.csv")

Comenzando procesamiento
procesando un 0.0%
procesando un 10.0%
procesando un 20.0%
procesando un 30.0%
procesando un 40.0%
procesando un 50.0%
procesando un 60.0%
procesando un 70.0%
procesando un 80.0%
procesando un 90.0%


In [12]:
print("Emiliano"[1:-1])

milian


In [30]:
separar_palabras(datos["dialogue"][0]), datos["dialogue"][0]

(['Está', 'saliendo', 'la', 'luna.', 'No', 'podemos', 'perder', 'tiempo.'],
 'Está saliendo la luna.\nNo podemos perder tiempo.')

In [9]:
datos_listos[(datos_listos["instancia_id"]==48)& (datos_listos["capitalizacion"]==3)], datos_preprocesado[47]

(      instancia_id token  token_id punt_inicial punt_final  capitalizacion
 1838            48     y       193         None       None               3,
 'Qué más da. Al fin y al cabo, es sólo tejido muerto.\nSólo podría engendrar el mal.\nSu salud se deteriorará si continúa con esta locura.\nEstoy asombrosamente cuerdo, doctor.\nHa creado un monstruo y éste le destruirá.\nPaciencia.\nCreo en este monstruo, como usted lo llama.\nY si usted no cree en él, será mejor que se marche.\nPiense en Elizabeth. En su padre.\nElizabeth cree en mí.\n¿Mi padre? Jamás cree en nadie.')

In [ ]:
datos_primer_decima_parte=datos["dialogue"].loc[:len(datos)//10]  #primer decimo me entra en memoria tarda una hora

datos_listos=asignar_labels(datos_primer_decima_parte,"datos_modelo.csv") #subtitulso

Comenzando procesamiento
procesando un 0.0%


In [6]:
datos_listos

NameError: name 'datos_listos' is not defined

In [11]:
separar_palabras(datos["dialogue"][128])

['Pareces', 'un', 'poco', 'nervioso.', 'Yo,', '¿nervioso.', '?', 'No,', 'yo.']

In [ ]:

def asignar_labels_fraccional(datos: pd.Series,inicio:int = 0)->pd.DataFrame:
   
    max_intervalos=int(1/prop):
    
    for i in range(inicio,max_intervalos):
        chunck=datos[i*len(datos)*prop,i+1*len(datos)*prop]
        chunk_procesado=asignar_labels(chunk,f"datos_modelo_{i}.csv")

    return chunck_procesado
        

In [ ]:
datos

In [ ]:
from datasets import load_datas/

num_muestras = 1000

print(f"Cargando {num_muestras} muestras en 'modo streaming'")

try:
    dataset_iterable = load_dataset(
        path="openbmb/DCAD-2000",
        name="spa_Latn",
        split="train",
        streaming=True
    )

    print("\nDataset listo en modo streaming.")
    muestras_tomadas = dataset_iterable.take(num_muestras)
    lista_de_muestras = list(muestras_tomadas)

except Exception as e:
    print(f"\nError al cargar el dataset en modo streaming: {e}")

### Visualizar de Muestras

In [ ]:

print(f"--- Visualizando las {len(lista_de_muestras)} muestras descargadas ---")

for i, muestra in enumerate(lista_de_muestras):

    #print(muestra)

    if 'text' in muestra:
        a = 2
        print(f"\n[Muestra {i + 1}]")
        print(muestra['text'])
    else:
        print(f"\n[Muestra {i + 1}] No se encontró la clave 'text'.")
        print("Contenido de la muestra:")
        print(muestra)

print("\n--- Fin de la visualización ---")

In [ ]:
def conseguir_palabras(texto, puntuacion_a_ignorar='()¡!:;…"'):
    # quitar la puntuacion que no nos interesa predecir.
    # Podria haber razones para decidir reemplazar un ... por . o hacer que los signos de exclamacion pongan mayusculas
    # A priori, no invente reglas asi.

    # Quizas si venga bien poner una regla que diga reemplazar '(' y ')' por ', '
    translator = str.maketrans('', '', puntuacion_a_ignorar)
    texto_con_puntuacion_a_adivinar = texto.translate(translator)
    
    # Me gusto la idea de usar regex
    texto_con_puntuacion_a_adivinar = re.sub(r'\.+', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'\?+', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r',+', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿+', '¿', texto_con_puntuacion_a_adivinar)

    # Evitar que hola . se separe en palabras 'hola' y '.'. Lo mismo para el resto de las puntuaciones
    texto_con_puntuacion_a_adivinar = re.sub(r' +\.', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +,', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +\?', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿ +', '¿', texto_con_puntuacion_a_adivinar)

    # Sacar saltos de linea y otros tipos de cosas (Desconozco la lista completa)
    texto_monoespaciado = re.sub(r'\s+', ' ', texto_con_puntuacion_a_adivinar).strip()
    palabras = texto_monoespaciado.split()
    return palabras

def transformar_input(texto):
    palabras_y_puntuaciones = conseguir_palabras(texto)
    palabras = []

    translator = str.maketrans('', '', '¿?,.')

    for palabra in palabras_y_puntuaciones:
        abre_pregunta = '¿' in palabra
        cierra_pregunta = '?' in palabra
        continua_texto = ',' in palabra
        cierra_texto = '.' in palabra
        valor_puntuaciones = [abre_pregunta, cierra_pregunta, continua_texto, cierra_texto]

        palabra_sin_puntuacion = palabra.translate(translator)
        palabras.append([palabra_sin_puntuacion, valor_puntuaciones])
    
    return palabras

print(lista_de_muestras[0].get('text'))
print()
asd = transformar_input(lista_de_muestras[0].get('text'))
asd

# Permitimos que haya mas de una puntuacion al inicio o al final?
#asd = conseguir_palabras("¿     Hola,    ?.")
#asd == ['¿Hola,?.']

# Que hacemos con estos casos?
#dsa = conseguir_palabras("asd.asd")
#dsa == ['asd.asd']

In [ ]:
import re
import torch
from transformers import BertTokenizer, BertModel

textos_originales= []
textos_aplanados = []
PUNT_TODOS = '¿¡.,?!:;…"'

for i, muestra in enumerate(lista_de_muestras):

    # Obtener texto_original (usando la clave 'text')
    texto_original = muestra.get('text')
    textos_originales.append(texto_original)
    if not texto_original:
        print(f"Saltando muestra {i}, clave 'text' no encontrada o vacía.")
        continue

    # Crear texto_plano (todo minúsculas, sin puntuación)
    texto_lower = texto_original.lower()
    # quitar toda la puntuación
    translator = str.maketrans('', '', PUNT_TODOS)
    texto_plano = texto_lower.translate(translator)
    # Reemplazar múltiples espacios por uno
    texto_plano = re.sub(r'\s+', ' ', texto_plano).strip()
    textos_aplanados.append(texto_plano)


In [ ]:
textos_originales[0]

In [ ]:
textos_aplanados[0]